# **Hybrid Image–Text Evaluation Notebook**

This Colab‑style notebook demonstrates a **hybrid evaluation pipeline** that combines:

1. A **rule‑based scorer** (`evaluar_reglas`) for visual, spatial and technical keywords  
2. A **CLIP‑based similarity** (`evaluar_clip`) using OpenCLIP  

Together, these yield a 0–5 “rule” score in each category plus a CLIPScore (ideal > 0.3), for a total out of 20.

---

## 1. Setup & Dependencies

In [ ]:
# Install OpenCLIP from source and other requirements
pip install git+https://github.com/mlfoundations/open_clip.git
pip install torch pillow

In [ ]:
# Imports
import open_clip
import torch
from PIL import Image
import re

#2. Rule-Based Scoring: evaluar_reglas

High‑level: scan a text description for keywords in three dimensions—visual, espacial, técnica—and cap each at 5 points.

In [ ]:
def evaluar_reglas(texto):
    puntaje = {"visual": 0, "espacial": 0, "tecnica": 0}

    # ─ Visual ─
    # Colores básicos y extendidos
    if re.search(r"\b(red|green|blue|yellow|black|white|orange|purple|cyan|magenta|gray|grey|brown)\b", texto, re.I):
        puntaje["visual"] += 1
    # Formas y símbolos
    if re.search(r"\b(curve|line|dotted|dashed|solid|circle|square|triangle|polygon|ellipse|symbol|marker)\b", texto, re.I):
        puntaje["visual"] += 1
    # Estilos, texturas y efectos
    if re.search(r"\b(gradient|texture|shading|contrast|brightness|saturation|hue|opacity|transparency|shadow|outline|border|fill)\b", texto, re.I):
        puntaje["visual"] += 1
    # Patrones y repeticiones
    if re.search(r"\b(pattern|repetition|cluster|grid|tiling|mosaic)\b", texto, re.I):
        puntaje["visual"] += 1
    # Composición y balance
    if re.search(r"\b(symmetry|asymmetry|balance|proportion|alignment)\b", texto, re.I):
        puntaje["visual"] += 1
    # Tope a 5 puntos
    if puntaje["visual"] > 4:
        puntaje["visual"] = 5

    # ─ Espacial ─
    # Ejes y leyendas (peso 2)
    if re.search(r"\b(x-axis|y-axis|axis|legend|inset|tick|tick mark|gridline|coordinate)\b", texto, re.I):
        puntaje["espacial"] += 2
    # Dirección y posición (peso 1)
    if re.search(r"\b(top|bottom|left|right|center|corner|north|south|east|west|up|down|inner|outer)\b", texto, re.I):
        puntaje["espacial"] += 1
    # Escalas y rangos (peso 1)
    if re.search(r"\b(range|scale|step|increment|zoom|viewport)\b", texto, re.I):
        puntaje["espacial"] += 1
    # Disposición y layout (peso 1)
    if re.search(r"\b(layout|panel|subplot|facet|section|margin|padding|spacing|gap|proximity|adjacent|overlap)\b", texto, re.I):
        puntaje["espacial"] += 1
    # Capas y profundidad (peso 1)
    if re.search(r"\b(layer|depth|z-index|stack|foreground|background)\b", texto, re.I):
        puntaje["espacial"] += 1
    # Tope a 5 puntos
    if puntaje["espacial"] > 5:
        puntaje["espacial"] = 5

    # ─ Técnica ─
    # Métodos y enfoques generales (peso 2)
    if re.search(r"\b(method|methodology|approach|framework|protocol|design|architecture|strategy)\b", texto, re.I):
        puntaje["tecnica"] += 2
    # Modelos, simulaciones, experimentos y análisis (peso 2)
    if re.search(r"\b(algorithm|model(?:ing)?|simulation|experiment|analysis|evaluation|mechanism|optimization|filtering|dynamics)\b", texto, re.I):
        puntaje["tecnica"] += 2
    # Matemáticas y formalismos (peso 2)
    if re.search(r"\b(equation|theorem|proof|derivative|integral|function|gradient|statistic|matrix|vector|eigenvalue|probability|distribution|Fourier|transform)\b", texto, re.I):
        puntaje["tecnica"] += 2
    # Datos, métricas e indicadores (peso 1)
    if re.search(r"\b(data|variable|parameter|result|finding|observation|measurement|metric|indicator)\b", texto, re.I):
        puntaje["tecnica"] += 1
    # Biología y ciencias de la vida (peso 1)
    if re.search(r"\b(cell|organism|protein|genome|DNA|RNA|enzyme|metabolism|photosynthesis|ecology|taxonomy)\b", texto, re.I):
        puntaje["tecnica"] += 1
    # Química y materiales (peso 1)
    if re.search(r"\b(molecule|compound|reaction|catalyst|polymer|stoichiometry|pH|titration|thermodynamics|acid|base|spectroscopy)\b", texto, re.I):
        puntaje["tecnica"] += 1
    # Física y mecánica (peso 1)
    if re.search(r"\b(force|stress|strain|load|energy|field|particle|wave|quantum|entropy|momentum|inertia|relativity|thermodynamics)\b", texto, re.I):
        puntaje["tecnica"] += 1
    # Ingeniería eléctrica y electrónica (peso 1)
    if re.search(r"\b(circuit|voltage|current|resistor|capacitor|inductor|signal|waveform|DSP|filter)\b", texto, re.I):
        puntaje["tecnica"] += 1
    # Informática y TI (peso 1)
    if re.search(r"\b(software|hardware|network|database|API|cloud|container|virtualization|microservice|encryption|cybersecurity|blockchain)\b", texto, re.I):
        puntaje["tecnica"] += 1
    # Inteligencia artificial y aprendizaje automático (peso 1)
    if re.search(r"\b(AI|ML|deep learning|reinforcement learning|neural network|NLP|computer vision|CV|clustering|regression|classification)\b", texto, re.I):
        puntaje["tecnica"] += 1
    # Ciencias sociales (peso 1)
    if re.search(r"\b(survey|case study|interview|ethnography|qualitative|quantitative|discourse|contextual|source analysis)\b", texto, re.I):
        puntaje["tecnica"] += 1
    # Historia y humanidades (peso 1)
    if re.search(r"\b(chronology|artifact|primary source|secondary source|historiography|archive|archival|periodization)\b", texto, re.I):
        puntaje["tecnica"] += 1
    # Economía y finanzas (peso 1)
    if re.search(r"\b(KPI|ROI|NPV|IRR|cash flow|financial model|balance sheet|economics)\b", texto, re.I):
        puntaje["tecnica"] += 1
    # Ingeniería mecánica y civil (peso 1)
    if re.search(r"\b(torque|fluid|solid mechanics|heat transfer|dynamics|stress analysis|fatigue)\b", texto, re.I):
        puntaje["tecnica"] += 1
    # Síntesis, caracterización y pruebas (peso 1)
    if re.search(r"\b(synthesis|characterization|fabrication|testing|benchmark)\b", texto, re.I):
        puntaje["tecnica"] += 1
    # Tope a 5 puntos
    if puntaje["tecnica"] > 5:
        puntaje["tecnica"] = 5


    return puntaje

# 3. CLIP‑Based Scoring: evaluar_clip

High‑level: load a ViT‑H‑14 model, embed image & text, normalize, then compute cosine similarity.

In [ ]:
def evaluar_clip(texto, image_path, model_name="ViT-H-14", pretrained="laion2b_s32b_b79k"):
    model, _, preprocess = open_clip.create_model_and_transforms(model_name, pretrained=pretrained)
    tokenizer = open_clip.get_tokenizer(model_name)
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model = model.to(device)

    image = Image.open(image_path).convert("RGB")
    image_input = preprocess(image).unsqueeze(0).to(device)
    text_input = tokenizer([texto]).to(device)

    with torch.no_grad():
        img_feat = model.encode_image(image_input)
        txt_feat = model.encode_text(text_input)
        img_feat /= img_feat.norm(dim=-1, keepdim=True)
        txt_feat /= txt_feat.norm(dim=-1, keepdim=True)
        return (img_feat @ txt_feat.T).item()

#4. Usage Example

In [ ]:
# ──────────────────────────────────────────────────────────────
# INGRESO DE DESCRIPCIÓN E IMAGEN A CALIFICAR
# ──────────────────────────────────────────────────────────────
descripcion = """The image presents a graph illustrating the Bit Error Rate (BER) versus Received Power in dBm, comparing the performance of PAM4-50Gbps-9dB and QPSK-50Gbps-9dB with DSP. The red curve, representing QPSK with DSP, consistently shows a significantly lower BER at equivalent received power levels compared to the green curve representing PAM4, indicating superior error performance for the QPSK system. Accompanying the graph are two inset images: a QPSK constellation diagram, visually confirming the four distinct phase states of the QPSK signal, and a PAM4 eye diagram, illustrating the four amplitude levels characteristic of PAM4 signaling, with arrows linking these diagrams to their respective performance curves on the main plot."""
ruta_imagen = "/Users/josu/Documents/TIC/CLIPSCORE/im.PNG"

# ──────────────────────────────────────────────────────────────
# Calculo
# ──────────────────────────────────────────────────────────────
p_reglas = evaluar_reglas(descripcion)
p_clip = evaluar_clip(descripcion, ruta_imagen)

# ─ Mostrar resultados
print("Evaluación híbrida:")
print(f"CLIPScore (OpenCLIP):         {p_clip:.4f} (rango ideal > 0.3)")
print(f"Visual (color/forma):         {p_reglas['visual']} / 5")
print(f"Disposición (layout):         {p_reglas['espacial']} / 5")
print(f"Técnica (contenido experto):  {p_reglas['tecnica']} / 5")
print(f"PUNTAJE TOTAL:                {(p_clip*5 + sum(p_reglas.values())):.2f} / 20.0 (CLIP pondera 25%)")


#5. Results

DESCRIPTA.IA

In [ ]:
(venv) josu@MacBook-Air-de-Josu TIC % /Users/josu/Documents/TIC/IMAGES/venv/bin/python /Users/josu/Documents/TIC/CLIPSCORE/pun.py
/Users/josu/Documents/TIC/IMAGES/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
Evaluación híbrida:
CLIPScore (OpenCLIP):         0.3727 (rango ideal > 0.3)
Visual (color/forma):         4 / 5
Disposición (layout):         5 / 5
Técnica (contenido experto):  3 / 5
PUNTAJE TOTAL:                13.86 / 20.0 (CLIP pondera 25%)

ChatGPT

In [ ]:
(venv) josu@MacBook-Air-de-Josu TIC % /Users/josu/Documents/TIC/IMAGES/venv/bin/python /Users/josu/Documents/TIC/CLIPSCORE/pun.py
/Users/josu/Documents/TIC/IMAGES/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
Evaluación híbrida:
CLIPScore (OpenCLIP):         0.3870 (rango ideal > 0.3)
Visual (color/forma):         2 / 5
Disposición (layout):         0 / 5
Técnica (contenido experto):  1 / 5
PUNTAJE TOTAL:                4.94 / 20.0 (CLIP pondera 25%)

Grok3

In [ ]:
(venv) josu@MacBook-Air-de-Josu TIC % /Users/josu/Documents/TIC/IMAGES/venv/bin/python /Users/josu/Documents/TIC/CLIPSCORE/pun.py
/Users/josu/Documents/TIC/IMAGES/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
Evaluación híbrida:
CLIPScore (OpenCLIP):         0.4068 (rango ideal > 0.3)
Visual (color/forma):         2 / 5
Disposición (layout):         3 / 5
Técnica (contenido experto):  1 / 5
PUNTAJE TOTAL:                8.03 / 20.0 (CLIP pondera 25%)

Gemini

In [ ]:
(venv) josu@MacBook-Air-de-Josu TIC % /Users/josu/Documents/TIC/IMAGES/venv/bin/python /Users/josu/Documents/TIC/CLIPSCORE/pun.py
/Users/josu/Documents/TIC/IMAGES/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
Evaluación híbrida:
CLIPScore (OpenCLIP):         0.3970 (rango ideal > 0.3)
Visual (color/forma):         2 / 5
Disposición (layout):         2 / 5
Técnica (contenido experto):  1 / 5
PUNTAJE TOTAL:                6.98 / 20.0 (CLIP pondera 25%)